## Steenbras Dam, South Africa

**Station Name:** sbas

**Latitude:** -34.18704704

**Longitude:** 18.84986166

**Ellipsoidal Height(m):** 403.342

**Network:** TRIGNET

**Archive:** EarthScope Consortium (use the special option)

<img src=https://gnss-reflections.org/static/images/SBAS.jpg width=500>


The data used here have been kindly provided by TRIGNET so that we could access L1 and L2C signals 
at 15 second sampling. This sampling rate is needed to be able to see the reservoir. However, they are in a proprietary Trimble binary format.  
It is non-trivial to convert those to RINEX, so in this example we will rely on files translated and 
archived by UNAVCO for this use case example. They can be used by specifying the "special" archive.
Here we will only discuss GPS - I would encourage you to also look at the Glonass data.

In [ ]:
from gnssrefl.daily_avg_cl import daily_avg
from gnssrefl.gnssir_input import make_gnssir_input
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.quickLook_cl import quicklook
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.installexe_cl import installexe
import ipywidgets as widgets
import os

from pathlib import Path

from gnssrefl.utils import check_environment, set_environment, get_sys

notebook_dir = Path.cwd().parents[1]

# Making sure environment variables are set - this is required to run the gnssrefl code
exists = check_environment()
if exists == False:
    set_environment(
        refl_code=str(notebook_dir),
        orbits=str(notebook_dir / "orbits"),
        exe=str(notebook_dir / "exe"),
    )

# If you are running this locally - make sure the items in the exe folder have execution permissions

# Set local variable of refl_code location
refl_code_loc = os.environ["REFL_CODE"]
print("refl_code location:", refl_code_loc)

# import gnssrefl functions


# import the crx2rnx file which is dependent on your working OS - this is required to run the gnssrefl code
# If in docker environment, then you do not need to download crxnrnx
try:
    os.environ["DOCKER"]
except KeyError:
    sys = get_sys()
    installexe(sys)

**Fast Mode:** 
this will download data weekly instead of daily

In [ ]:
weekly = widgets.Checkbox(
    value=True, description="Fast Mode", disabled=False, indent=False
)
weekly

Deselect the Fast Mode checkbox to deactivate fast mode.

### Reflection Zones 

Use the [reflection zone web app](https://gnss-reflections.org/rzones) to think about which azimuths and elevation angles to use.
Note the photograph!  **You are not 6 meters (vertically) above the water.** You will not be able to fully test
the reflector height value used in this web app until you look at some real data. But you can get an idea of which
RH values put you over the water.

### Evaluate the Data

We know that the reservoir will only be in the northeast quadrant, so I am going 
to select those azimuths specifically. I will start with elevation angle limits of 5-12 degrees and the superior L2C frequency.

In [ ]:
station = "sbas"

rinex2snr(station, 2020, 1, archive="special")

Lets try different heights:

In [ ]:
vals, metrics = quicklook(
    station, 2020, 1, e2=12, h1=0.5, h2=10, azim1=0, azim2=75, fr=20
)

In [ ]:
vals, metrics = quicklook(
    station, 2020, 1, e2=12, h1=10, h2=20, azim1=0, azim2=75, fr=20
)

In [ ]:
vals, metrics = quicklook(station, 2020, 1, e2=12, h1=20, h2=35, fr=20)

You can add the screenstats parameter to get more details about the plot values.

In [ ]:
vals, metrics = quicklook(
    station, 2020, 1, e2=12, h1=20, h2=35, azim1=0, azim2=90, fr=20, screenstats=True
)

Looks like satellite 5 at an azimuth of 78.4 degrees is the problem, so we 
will further restrict our analysis to 78 degrees in the next section.

You might go back and use the reflection zone web app to make sure that your azimuth and 
elevation angles look good for a RH of 28 meters.


### Estimate Lake Level

Make the SNR files:

In [ ]:
rinex2snr(station, 2020, 1, doy_end=366,
          archive="special", weekly=weekly.value)

Save your analysis strategy. restrict azimuths to 0-78 degrees.

In [ ]:
make_gnssir_input(station, e2=12, h1=20, h2=35,
                  peak2noise=3, ampl=10, azlist2=[0, 78])

Hand edit the json file to restrict azimuths to 0-78 degrees and include 340-360.


The next step is to estimate reflector heights. First do a single day using the plt option.

In [ ]:
gnssir("sbas", 2020, 1, plt=True)

For the L1 frequecy reflector height plot - you can see what happens if you violate the Nyquist - double peaks!

Luckily this is not prevalent in this dataset thanks to the help of TRIGNET 
using a 15 second sample rate in the files.

You'll notice that the L2C frequency - which has a longer wavelength than L1 - does not have a double peak.
And that is what we should expect.

Go ahead and estimate reflector heights for all available days:

In [ ]:
gnssir(station, 2020, 1, doy_end=365)

Compute a daily average. Since we only have reflections in one geographic quadrant and are 
only using GPS signals, we should not require as many points as we have done in other examples. 
I am going to start by requiring 10:

In [ ]:
daily_avg(station, 0.25, 10)

Note the missing points - this has nothing to do with reflectometry. These are days when the station did not report data.
Numerical values are saved in a file. The location of the file is printed to the screen.

You should also go back and add some of those very northwestern tracks by setting a second azimuth region in your json file.
Then rerun the <code>gnssir</code> and <code>daily_avg</code> codes.

### Compare with in situ data:

[Current state of the lower Steenbras reservoir](https://www.dws.gov.za/Hydrology/Weekly/percentile.aspx?station=%20G4R001)

[This is a link for all reservoirs](https://www.dws.gov.za/Hydrology/Weekly/ProvinceWeek.aspx?region=WC)

Simon Williams found this web app that will [provide 2020 data for a comparison](https://www.dws.gov.za/Hydrology/Verified/HyData.aspx?Station=G4R001100.00&DataType=Point&StartDT=2020-01-01&EndDT=2020-12-31&SiteType=RES)

As a final remark, there are lots of lake/reservoir/river gauges around the world. The beauty of using this GPS dataset to measure 
the reservoir level is that you can combine these RH values with the [height estimates from "normal" GPS](http://geodesy.unr.edu/NGLStationPages/stations/SBAS.sta).
This means your reservoir measurements are in ITRF and that is something *no other in situ lake level measuring system can do.*